In [8]:
import pandas as pd
import urllib.request
import re
import datetime
import itertools

In [16]:
url = 'http://travel.mtanyct.info/serviceadvisory/routeStatusResult.aspx?tag=ALL&date=11/17/2017&time=&method=getstatus4'
data = get_data(url)
data = extract(data)
#data = drop_bi(data)
def split_note(data):
    return list(map(lambda x: x.split('<BR CLEAR=LEFT>'), data))[1:]
data = split_note(data)
# data = drop(data)
# data = rep_ls(data)
# data = drop_ff(data,"F&F_Pos_16 (LINE)")
# data = drop_ff(data,"&nbsp;")
# data = drop_ff(data,"\\")
data

[['" onclick=ShowHide(1);><b><i>TRACK REPLACEMENT</i><br clear=left><IMG src=\\\'images/1.png\\\' > Van Cortlandt Park-bound trains skip<font color=#000000> 50 St, 59 St and 66</font> St'],
 ['" onclick=ShowHide(2);><b><i>ELECTRICAL IMPROVEMENTS</i><br clear=left><font color=#000000><IMG src=\\\'images/1.png\\\' > South Ferry-bound trains skip 86, 79, 66, 59 and 50 Sts'],
 ['" onclick=ShowHide(3);><font color=#000000><b><i>TRACK MAINTENANCE</i>',
  "<IMG src=\\'images/2.png\\' > Flatbush Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy"],
 ['" onclick=ShowHide(4);><b><i>ELECTRICAL IMPROVEMENTS</i><br clear=left><font color=#000000><IMG src=\\\'images/2.png\\\' > Flatbush Av-bound trains skip 86, 79, 66, 59 and 50 Sts'],
 ['" onclick=ShowHide(5);><font color=#000000><b><i>TRACK MAINTENANCE</i><br clear=left><IMG src=\\\'images/2.png\\\' > Flatbush Av-bound trains run local from 34 St-Penn Station to Chambers St'],
 ['" onclick=ShowHide(6);><font color=#000000><b><i>TRAC

In [12]:
def get_data(ur1):
    return str(urllib.request.urlopen(ur1).read())

def extract(data):
    result = data.split("underline")
    result = list(map(lambda x: x.split('</a><br/>')[0].split('<BR CLEAR=LEFT>') , result))[1:]
    return result

def drop_bi(data):
    data = list(map(lambda x: re.sub('<b><i>(.*?)</i>', '<b>',x) , data))
    return data

def drop(data):
    data = list(map(lambda x: x.split('<b>')[1] , data))
    return data

def rep(s):
    a= s.replace("<IMG src=\\'images/",' ').replace('6d', '6').replace(".png\\' >",'_LINE')
    
    return re.sub('<[^>]+>', '', a)

def rep_ls(ls):
    return list(map(rep, ls))

def drop_ff(ls, string):
    return list(map(lambda x: x.replace(string, ''), ls))

def clean(url):
    data = get_data(url)
    data = extract(data)
    data = drop_bi(data)
    data = drop(data)
    data = rep_ls(data)
    data = drop_ff(data,"F&F_Pos_16 (LINE)")
    data = drop_ff(data,"&nbsp;")
    data = drop_ff(data,"\\")
    return data

In [13]:
def get_clean_data(data):
    url = 'http://travel.mtanyct.info/serviceadvisory/routeStatusResult.aspx?tag=ALL&date='+data+'&time=&method=getstatus4'
    return clean(url)

In [14]:
get_clean_data('11/17/2017')

TypeError: expected string or bytes-like object

In [8]:
def get_data_over_period(start, num_of_days):
    start_date = datetime.datetime.strptime(start, '%m/%d/20%y')
    date_list = [start_date + datetime.timedelta(days=x) for x in range(0, num_of_days)]
    dates = list(map(lambda x: x.strftime('%m/%d/20%y'), date_list))
    dataset = []
    dataset = list(map(get_clean_data, dates))
    answer = set(list(list(itertools.chain.from_iterable(dataset))))
    return answer

In [9]:
n = get_data_over_period('11/7/2017', 5)

In [7]:
# text_file = open("Output.txt", "w")
# for s in n:
#     text_file.write(s+"\n")
# text_file.close()

In [10]:
n

{' 1_LINE Van Cortlandt Park-bound trains skip 50 St, 59 St and 66 St',
 ' 1_LINE Van Cortlandt Park-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE Flatbush Av-bound trains run local from 96 St to Times Sq-42 St',
 ' 2_LINE Flatbush Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 2_LINE Uptown trains skip 50 St, 59 St and 66 St',
 ' 2_LINE Wakefield-bound trains skip 79 St and 86 St 1_LINE trains provide alternate service',
 ' 2_LINE Wakefield-bound trains skip Franklin, Canal, Houston, Christopher, 18, 23 and 28 Sts',
 ' 2_LINE trains replace weekend  5_LINE service between Dyre Av and E 180 St',
 ' 3_LINE New Lots Av-bound trains run local from 96 St to Times Sq-42 St',
 ' 3_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 3_LINE No trains running',
 ' 4_LINE New Lots Av-bound trains skip Bergen St, Grand Army Plaza and Eastern Pkwy',
 ' 4_LINE Service operates every 16 minutes between 